## Reading the DHS survey and GPS datasets

Importing necessary libraries

In [14]:
from geopandas import read_file
import pandas as pd

Reading the shapefile to understand cluster locations

In [35]:
shapefile_df = read_file("../poverty_predictor/data/GPS/RWGE61FL.shp")

In [36]:
location_df = shapefile_df[['DHSCLUST', 'LATNUM', 'LONGNUM']]

In [37]:
location_df.head()

,DHSCLUST,LATNUM,LONGNUM
0,1.0,-2.532818,29.684726
1,2.0,-1.833858,30.310689
2,3.0,-1.888155,29.478298
3,4.0,-2.366763,30.521692
4,5.0,-2.171266,30.018541


Reading the survey (.dta/stata) file to understand the wealth asset data

In [106]:
dta_file = pd.read_stata('../poverty_predictor/data/Survey/RWHR61FL.dta')
survey_df = dta_file[['hv001', 'hv270']].rename(columns={'hv001': 'DHSCLUST', 'hv270':'wealth_asset_index'})
survey_df['wealth_asset_index'] = survey_df['wealth_asset_index'].map({'poorest':1, 'poorer':2, 'middle':3, 'richer':4, 'richest':5})
survey_df['wealth_asset_index'] = survey_df['wealth_asset_index'].astype('int16')

In [111]:
survey_df = survey_df.groupby('DHSCLUST')[['wealth_asset_index']].median().reset_index()

In [112]:
survey_df.head()

,DHSCLUST,wealth_asset_index
0,1,2.0
1,2,2.5
2,3,2.0
3,4,2.0
4,5,2.0


Merging location_df and survey_df to get wealth_df

In [113]:
wealth_df = pd.merge(location_df, survey_df, on = 'DHSCLUST')

In [114]:
wealth_df

,DHSCLUST,LATNUM,LONGNUM,wealth_asset_index
0,1.0,-2.532818,29.684726,2.0
1,2.0,-1.833858,30.310689,2.5
2,3.0,-1.888155,29.478298,2.0
3,4.0,-2.366763,30.521692,2.0
4,5.0,-2.171266,30.018541,2.0
...,...,...,...,...
487,488.0,-2.389778,29.451722,2.0
488,489.0,-2.144317,30.348895,3.0
489,490.0,-1.967830,30.441541,5.0
490,491.0,-1.441294,30.063765,3.0
